In [ ]:
#coding: utf-8
#source activate py39

import pandas as pd
import numpy as np

source_data = '../collect/binance_history/ETHUSDT_binance.csv'
df = pd.read_csv(source_data)
df['date'] = pd.to_datetime(df['Time']/1000, unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
df['max_low_chg'] = df['low']/df['open'] -1
df['next_24h_chg'] = ((df['close'].shift(-24) - df['close']) / df['close']) * 100

In [ ]:
'''
TODO:
- exit à certains %
- exit à certains horizons de temps
'''

In [55]:
import random

def generate_combinations(num_combinations, min_size, max_size, min_level, max_level):
    combinations = []
    
    for _ in range(num_combinations):
        current_size = random.randint(min_size, max_size)
        current_list = []
        remaining_prorata = 1.0

        for i in range(current_size - 1):  # Générer un élément de moins pour contrôler la somme des prorata à la fin
            # S'assurer que le prorata généré est > 0 en laissant suffisamment pour les éléments restants
            prorata = round(random.uniform(0.01, remaining_prorata - (0.01 * (current_size - i - 1))), 2)
            remaining_prorata -= prorata
            level = round(random.uniform(min_level, max_level), 2)
            current_list.append({'level': level, 'prorata': prorata})

        # Ajouter le dernier élément avec le prorata restant pour assurer que la somme soit exactement 1
        level = round(random.uniform(min_level, max_level), 2)
        current_list.append({'level': level, 'prorata': round(remaining_prorata, 2)})

        combinations.append(current_list)
    
    return combinations

# Générer 1000 combinaisons respectant les conditions données
generated_combinations = generate_combinations(1000, 3, 10, -0.5, -0.03)

# Affichage des 5 premières combinaisons pour vérification
for i, combination in enumerate(generated_combinations[:5]):
    print(f"Combinaison {i+1}: {combination}")


Combinaison 1: [{'level': -0.1, 'prorata': 0.44}, {'level': -0.09, 'prorata': 0.08}, {'level': -0.48, 'prorata': 0.4}, {'level': -0.27, 'prorata': 0.01}, {'level': -0.3, 'prorata': 0.03}, {'level': -0.05, 'prorata': 0.04}]
Combinaison 2: [{'level': -0.4, 'prorata': 0.06}, {'level': -0.29, 'prorata': 0.08}, {'level': -0.31, 'prorata': 0.58}, {'level': -0.25, 'prorata': 0.16}, {'level': -0.05, 'prorata': 0.07}, {'level': -0.31, 'prorata': 0.01}, {'level': -0.28, 'prorata': 0.02}, {'level': -0.3, 'prorata': 0.01}, {'level': -0.22, 'prorata': 0.01}]
Combinaison 3: [{'level': -0.42, 'prorata': 0.73}, {'level': -0.19, 'prorata': 0.16}, {'level': -0.45, 'prorata': 0.01}, {'level': -0.28, 'prorata': 0.1}]
Combinaison 4: [{'level': -0.07, 'prorata': 0.85}, {'level': -0.23, 'prorata': 0.08}, {'level': -0.23, 'prorata': 0.01}, {'level': -0.04, 'prorata': 0.01}, {'level': -0.17, 'prorata': 0.01}, {'level': -0.36, 'prorata': 0.01}, {'level': -0.09, 'prorata': 0.01}, {'level': -0.21, 'prorata': 0.01

In [66]:
generated_combinations = [
    [{'level': -0.03, 'prorata': 0.71}, {'level': -0.4, 'prorata': 0.22}, {'level': -0.10, 'prorata': 0.02}],
    [{'level': -0.03, 'prorata': 0.5}, {'level': -0.4, 'prorata': 0.5}, {'level': -0.10, 'prorata': 0.02}],
    [{'level': -0.03, 'prorata': 0.6}, {'level': -0.4, 'prorata': 0.2}, {'level': -0.10, 'prorata': 0.2}],
    [{'level': -0.03, 'prorata': 0.3}, {'level': -0.5, 'prorata': 0.3}, {'level': -0.07, 'prorata': 0.4}],
    [{'level': -0.03, 'prorata': 0.5}, {'level': -0.5, 'prorata': 0.2}, {'level': -0.07, 'prorata': 0.2}, {'level': -0.1, 'prorata': 0.2}]
]

In [63]:

max_risk = 10000  # Montant que je suis prêt à perdre au maximum
liq = 0.5  # Niveau auquel je suis liquidé

df_trades = pd.DataFrame(columns=["date", "pnl", "brokie"])
df_results = pd.DataFrame(columns=["combination", "pnl", "n_sl", "n_trades"])

for combination in generated_combinations:
    print(combination)
    in_trade = 0
    avg_price = 0
    size = 0
    sl = 0
    entry_date = 0
    brokie = 0
    df_trades = df_trades.iloc[0:0]  # Réinitialiser df_trades pour chaque combinaison

    for index, row in df.iterrows():
        if row['max_low_chg'] < combination[0]['level'] and not in_trade:
            sl = row['open'] * (1 - liq)
            entry_date = row['date']
            for level in combination:
                if row['max_low_chg'] < level['level']:
                    new_position_size = max_risk * level['prorata'] / (row['open'] * (1 + level['level']) - sl)
                    if in_trade:
                        avg_price = ((avg_price * size) + (row['open'] * (1 + level['level']) * new_position_size)) / (size + new_position_size)
                    else:
                        in_trade = 1
                        avg_price = row['open'] * (1 + level['level'])
                    size += new_position_size
                    
        if in_trade:
            in_trade += 1
            if in_trade == 24 or row['low'] < sl:
                pnl = (row['close'] - avg_price) * size
                new_row = pd.DataFrame({"date": [entry_date], "pnl": [pnl], "brokie": [brokie]}, index=[0])
                df_trades = pd.concat([df_trades, new_row], ignore_index=True)
                in_trade = 0
                size = 0
                avg_price = 0
                brokie = 0 if row['low'] >= sl else 1

    new_row = pd.DataFrame({"combination": [str(combination)], "pnl": [df_trades['pnl'].sum()], "n_sl": [df_trades['brokie'].sum()], "n_trades": [df_trades.shape[0]]}, index=[0])
    df_results = pd.concat([df_results, new_row], ignore_index=True)

print(df_results)


[{'level': -0.03, 'prorata': 0.71}, {'level': -0.4, 'prorata': 0.22}, {'level': -0.1, 'prorata': 0.02}]
[{'level': -0.03, 'prorata': 0.5}, {'level': -0.4, 'prorata': 0.5}, {'level': -0.1, 'prorata': 0.02}]
[{'level': -0.03, 'prorata': 0.6}, {'level': -0.4, 'prorata': 0.2}, {'level': -0.1, 'prorata': 0.2}]
[{'level': -0.03, 'prorata': 0.3}, {'level': -0.5, 'prorata': 0.3}, {'level': -0.07, 'prorata': 0.4}]
[{'level': -0.03, 'prorata': 0.5}, {'level': -0.5, 'prorata': 0.2}, {'level': -0.07, 'prorata': 0.2}, {'level': -0.1, 'prorata': 0.2}]
                                         combination           pnl n_sl  \
0  [{'level': -0.03, 'prorata': 0.71}, {'level': ...  74220.918045    1   
1  [{'level': -0.03, 'prorata': 0.5}, {'level': -...  52317.688332    1   
2  [{'level': -0.03, 'prorata': 0.6}, {'level': -...  64252.070300    1   
3  [{'level': -0.03, 'prorata': 0.3}, {'level': -...  33360.477617    1   
4  [{'level': -0.03, 'prorata': 0.5}, {'level': -...  54857.035642    1   

  n_t

In [ ]:
#df.to_csv('ETHUSDT_binance_ready.csv', index=True)

In [64]:
print(df_results['pnl'].max())

74220.91804471251
